In [2]:
import json
import geojson
from PIL import Image
import piexif
import numpy as np
from collections import defaultdict
import shapefile
from shapely.geometry import Point, LineString
from rtree import index
import cv2


In [2]:
corners = [[-83.08489893270443, 30.123174014690044], [-83.08453339472855, 30.12318037458068], [-83.08452791372119, 30.122942291025208], [-83.08489345082155, 30.122935931191176]]

In [12]:
image_path = r"C:\Users\User\Downloads\copy_DJI_20240228083728_0005_D_aligned_xmp.tif"
output_path = r"C:\Users\User\Downloads\Aiman-file-results\temp"
image = cv2.imread(image_path)
image_height, image_width, _ = image.shape
top_left = (corners[1][1], corners[1][0])
bottom_right = (corners[3][1], corners[3][0])

In [10]:

def interpolate_to_gps(x, y):
    lon = top_left[1] + (x / image_width) * (bottom_right[1] - top_left[1])
    lat = top_left[0] + (y / image_height) * (bottom_right[0] - top_left[0])
    return lat, lon

In [15]:
x1, y1 = interpolate_to_gps(corners[0][0], corners[0][1])
x2, y2 = interpolate_to_gps(corners[1][0], corners[1][1])
x3, y3 = interpolate_to_gps(corners[2][0], corners[2][1])
x4, y4 = interpolate_to_gps(corners[3][0], corners[3][1])

In [17]:
import json

# Load your JSON file
json_path = r"C:\Users\User\Downloads\Aiman-file\DJI_20240228084011_0085_D.json"
with open(json_path, 'r') as file:
    data = json.load(file)

# Initialize variables to find the extents
min_x = float('inf')
min_y = float('inf')
max_x = float('-inf')
max_y = float('-inf')

# Iterate through the detections to find the minimum and maximum coordinates
for detection in data['detections']:
    box = detection['box']
    x1, y1 = box['x1'], box['y1']
    x2, y2 = box['x2'], box['y2']

    min_x = min(min_x, x1, x2)
    min_y = min(min_y, y1, y2)
    max_x = max(max_x, x1, x2)
    max_y = max(max_y, y1, y2)

# Print the extents
print(f"Top-left corner: ({min_x}, {min_y})")
print(f"Top-right corner: ({max_x}, {min_y})")
print(f"Bottom-left corner: ({min_x}, {max_y})")
print(f"Bottom-right corner: ({max_x}, {max_y})")


Top-left corner: (0.0, 0.0)
Top-right corner: (5280.0, 0.0)
Bottom-left corner: (0.0, 3956.0)
Bottom-right corner: (5280.0, 3956.0)


In [3]:
def get_angle(corners):
    """
    Calculate the correct rotation angle from corners
    """
    # Get the main edges
    edge1 = [corners[1][0] - corners[0][0], corners[1][1] - corners[0][1]]
    edge2 = [corners[3][0] - corners[0][0], corners[3][1] - corners[0][1]]
    
    # Calculate lengths
    len1 = np.sqrt(edge1[0]**2 + edge1[1]**2)
    len2 = np.sqrt(edge2[0]**2 + edge2[1]**2)
    
    # Use the longer edge for angle calculation
    if len1 > len2:
        dx, dy = edge1
    else:
        dx, dy = edge2
    
    angle = np.arctan2(dy, dx)
    return angle

corners = [[-83.08490226226006, 30.123817184960654], [-83.08453685334257, 30.12382354268575], [-83.08453137416528, 30.12358554475479], [-83.08489678220786, 30.123579187086264]]

print(get_angle(corners))

0.017397177146435655


In [20]:
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\User\Downloads\1880_image_details.csv")

In [5]:
df.head()

,image_name,corners,gsd,image_width,image_height
0,DJI_20250307112525_0066_D_CAMERA12_LA.JPG,"[[-83.16446549315401, 30.250503519065333], [-8...",0.000448,8064,6048
1,DJI_20250307112619_0067_D_CAMERA12_LA.JPG,"[[-83.1644820952984, 30.251570276647477], [-83...",0.000486,8064,6048
2,DJI_20250307112653_0068_D_CAMERA12_LA.JPG,"[[-83.16314248685867, 30.25303646118143], [-83...",0.000473,8064,6048
3,DJI_20250307112712_0069_D_CAMERA12_LA.JPG,"[[-83.16343240357156, 30.252448038825353], [-8...",0.000493,8064,6048
4,DJI_20250307112730_0070_D_CAMERA12_LA.JPG,"[[-83.1634283441402, 30.251552465436436], [-83...",0.000483,8064,6048


In [6]:
df2 = pd.read_csv(r"C:\Users\User\Downloads\1880_image_coordinates_new.csv", )

In [16]:
# a  = df2['elevation'] - df2['elevation'].iloc[0]
# b = df2['updated_elevations'] = df2['relative_altitude'] - df2['updated_elevations']
c = df2['gsd'] = (df2['sensor_width'] * df2['updated_elevations']) / (df2['focal_length'] * df2['width'])

In [18]:
df['gsd']  = c

In [19]:
df.head()

,image_name,corners,gsd,image_width,image_height
0,DJI_20250307112525_0066_D_CAMERA12_LA.JPG,"[[-83.16446549315401, 30.250503519065333], [-8...",0.000486,8064,6048
1,DJI_20250307112619_0067_D_CAMERA12_LA.JPG,"[[-83.1644820952984, 30.251570276647477], [-83...",0.000473,8064,6048
2,DJI_20250307112653_0068_D_CAMERA12_LA.JPG,"[[-83.16314248685867, 30.25303646118143], [-83...",0.000493,8064,6048
3,DJI_20250307112712_0069_D_CAMERA12_LA.JPG,"[[-83.16343240357156, 30.252448038825353], [-8...",0.000483,8064,6048
4,DJI_20250307112730_0070_D_CAMERA12_LA.JPG,"[[-83.1634283441402, 30.251552465436436], [-83...",0.000448,8064,6048


In [52]:
def gsd_correcter(cord_csv, image_csv, new_detail_csv):
    
    image_csv = pd.read_csv(image_csv)
    cord_csv = pd.read_csv(cord_csv)
    cord_csv.columns = cord_csv.columns.str.strip()
    image_csv.columns = image_csv.columns.str.strip()
    print(image_csv.columns)
    print(cord_csv.columns)

    # Rename column to match for merging
    cord_csv.rename(columns={'image': 'image_name'}, inplace=True)

    # Merge and update GSD values
    image_details_updated = image_csv.drop(columns=['gsd']).merge(
        cord_csv[['image_name', 'gsd']], on='image_name', how='left'
    )
    image_details_updated.to_csv(new_detail_csv, index=False)

In [53]:
gsd_correcter(r"C:\Users\User\Downloads\forest-rk\1906_image_coordinates_new.csv", 
              r"C:\Users\User\Downloads\forest-rk\1906_image_details.csv", 
              r"C:\Users\User\Downloads\forest-rk\1906_image_details_new.csv"
              )

Index(['image_name', 'corners', 'gsd', 'image_width', 'image_height'], dtype='object')
Index(['image', 'lat', 'lon', 'relative_altitude', 'camera_model', 'width',
       'height', 'sensor_width', 'focal_length', 'elevation', 'offset',
       'updated_elevations', 'gsd'],
      dtype='object')
